<a href="https://colab.research.google.com/github/wilberquito/NeuralNetworksCompetition2023/blob/main/Code/ML_STUDENT_COMPETITION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

In [2]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    from google.colab import data_table
    data_table.DataTable.max_columns = 50
    data_table.enable_dataframe_formatter()
else:
    from IPython.core.interactiveshell import InteractiveShell
    InteractiveShell.ast_node_interactivity = "all"

In [3]:
from pathlib import Path
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from pathlib import Path


pd.set_option('display.max_columns', 500)

torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [4]:
torch.cuda.is_available()

False

In [5]:
csv_path = Path('/content/drive/MyDrive/train.csv') if IN_COLAB else Path('../Data/train.csv')
df = pd.read_csv(csv_path)
df.head()

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,target
0,493553,0.315471,-0.183690,0.664383,-1.186794,0.665098,0.946208,0.729857,0,4,1,3,1,2,4,1,5,2,0,1,-3.690715,-0.628005,-2.832295,-1.409039,3.645067,0.233039,-3.754846,-1.061733,BDBBCACIBB,20.308715,1,0,0
1,237346,-1.286392,1.780592,0.576698,-2.690658,1.321997,-0.675894,0.371070,3,0,3,3,5,3,2,0,1,6,0,1,0.664517,-2.871912,3.826628,3.087653,0.494209,3.210875,-0.666457,0.123854,ACBDCBCADA,-449.291063,1,0,0
2,37368,-0.290303,-0.485907,0.808350,-0.156288,1.083632,-1.129914,0.767396,3,1,3,2,3,4,1,1,1,0,2,6,-0.494988,-0.946303,2.333223,2.084169,-4.782668,-1.671375,2.774382,2.273130,AABBABCLAF,-86.206118,0,1,1
3,665220,1.243590,0.035112,-1.013236,0.854267,0.019192,0.597892,-2.020416,2,0,4,5,0,5,1,0,3,1,1,3,-3.066427,-2.430158,-0.185332,-0.701691,-2.769142,-6.534231,-0.557677,-0.429972,ADBBABEEBA,-30.157403,0,2,1
4,41499,0.702716,0.817044,-0.064907,-1.045483,0.718374,0.164451,-0.936620,1,2,2,2,2,5,0,3,1,1,2,4,-1.899984,1.427460,-4.992610,1.154162,-1.931443,2.325042,2.143811,-1.039599,ABBBBBCMBB,296.484562,0,2,1


In [61]:
X, y = df.drop(['target', 'f_27', 'id'], axis=1), df['target']
X = (X - X.min()) / (X.max() - X.min())
X.shape, y.shape

((810000, 30), (810000,))

In [62]:
X.describe()

,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_28,f_29,f_30
count,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000
mean,0.492047,0.493137,0.483409,0.509344,0.489579,0.488666,0.500947,0.135472,0.128624,0.168756,0.155560,0.138772,0.177645,0.186609,0.108244,0.150097,0.139767,0.154813,0.158921,0.517581,0.487353,0.496712,0.510560,0.486064,0.465154,0.494703,0.538599,0.499857,0.345565,0.501186
std,0.106824,0.105223,0.104209,0.110100,0.103120,0.102858,0.103412,0.110450,0.099419,0.116962,0.117537,0.118276,0.110167,0.128143,0.097128,0.112068,0.103999,0.122292,0.120430,0.103392,0.105602,0.102610,0.105636,0.099971,0.100273,0.098855,0.091013,0.097074,0.475553,0.409425
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.419874,0.421923,0.413105,0.435033,0.419932,0.419381,0.431191,0.066667,0.062500,0.071429,0.071429,0.076923,0.125000,0.083333,0.000000,0.071429,0.066667,0.083333,0.076923,0.448633,0.415828,0.428052,0.440037,0.418822,0.397385,0.428586,0.479097,0.435201,0.000000,0.000000
50%,0.492222,0.493239,0.483497,0.509273,0.489467,0.488607,0.500879,0.133333,0.125000,0.142857,0.142857,0.153846,0.187500,0.166667,0.071429,0.142857,0.133333,0.166667,0.153846,0.518558,0.486864,0.496870,0.512280,0.485208,0.465171,0.494076,0.540384,0.499806,0.000000,0.500000
75%,0.564173,0.564118,0.553837,0.583519,0.559059,0.558114,0.570758,0.200000,0.187500,0.214286,0.214286,0.230769,0.250000,0.250000,0.142857,0.214286,0.200000,0.250000,0.230769,0.587767,0.558750,0.565416,0.582559,0.552316,0.532749,0.560920,0.600025,0.564667,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [63]:
y.value_counts()

0    415945
1    394055
Name: target, dtype: int64

In [64]:
type(X), type(y)

(pandas.core.frame.DataFrame, pandas.core.series.Series)

In [131]:
class NetV0(nn.Module):

    def __init__(self, input_features=30):
        super().__init__()
        self.main = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=64),
            nn.Tanh(),
            nn.Linear(in_features=64, out_features=64),
            nn.Tanh(),
            nn.Linear(in_features=64, out_features=64),
            nn.Tanh(),
            nn.Linear(in_features=64, out_features=64),
            nn.Tanh(),
            nn.Linear(in_features=64, out_features=128),
            nn.Tanh(),
            nn.Linear(in_features=128, out_features=128),
            nn.Tanh(),
            nn.Linear(in_features=128, out_features=128),
            nn.Tanh(),
            nn.Linear(in_features=128, out_features=64),
            nn.Tanh(),
            nn.Linear(in_features=64, out_features=64),
            nn.Tanh(),
            nn.Linear(in_features=64, out_features=64),
            nn.Tanh(),
            nn.Linear(in_features=64, out_features=64),
            nn.Tanh(),
            nn.Linear(in_features=64, out_features=1),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.main(x)

# Define a custom dataset by extending the PyTorch Dataset class
class CustomDataset(Dataset):

    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels 
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data[index]
        label = self.labels[index]
        if self.transform:
            item = self.transform(item)
        return item, label

In [132]:
# Declare the hole dataset
X_tensor, y_tensor = torch.from_numpy(X.to_numpy()), torch.from_numpy(y.to_numpy())
X_tensor, y_tensor = X_tensor.type(torch.float), y_tensor.type(torch.float)
dataset = CustomDataset(data=X_tensor, labels=y_tensor)
len(dataset)

810000

In [133]:
# Declare the train and test dataset
import numpy as np

test_size = 0.15
train_size = 1 - test_size

# Stratify by label
labels = np.array(y)
positive_indices = np.where(labels == 1)[0]
negative_indices = np.where(labels == 0)[0]

positive_split = int(train_size * len(positive_indices))
negative_split = int(train_size * len(negative_indices))

positive_train_indices = positive_indices[:positive_split]
positive_test_indices = positive_indices[positive_split:]
negative_train_indices = negative_indices[:negative_split]
negative_test_indices = negative_indices[negative_split:]

train_indices = np.concatenate([positive_train_indices, negative_train_indices])
test_indices = np.concatenate([positive_test_indices, negative_test_indices])

train_dataset = torch.utils.data.Subset(dataset, train_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

len(train_dataset), len(test_dataset)

(688499, 121501)

In [143]:
# Checkpoint
def checkpoint(epoch, model, optimizer, criteria, save_as: Path):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'criteria': criteria,
    }, save_as)
    

# Validate
def validation(model, device, valid_loader, criteria):
    # Settings
    model.eval()
    loss_total = 0
    accuracy_total = 0

    # Test validation data
    with torch.inference_mode():
        for inputs, labels in valid_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs).squeeze(dim=1)
            loss = criteria(outputs, labels)
            loss_total += loss.item() * inputs.size(0)
            matches = torch.round(torch.sigmoid(outputs)) == labels
            accuracy_total += (torch.sum(matches.to(torch.int32))).item()
                        
    return (loss_total / len(valid_loader), accuracy_total / len(valid_loader.dataset))


# Train
def train(device, model, epochs, optimizer, criteria, train_loader, valid_loader, resume=1, save_as=Path('../Output/model.pth')):
    # Early stopping
    last_loss = 100
    patience = 5
    trigger_times = 0
    
    train_loss_track, train_accuracy_track = [], []
    test_loss_track, test_accuracy_track = [], []

    # Move model to device
    model = model.to(device)

    for epoch in range(resume, epochs + 1):
        model.train()

        for times, (inputs, labels) in enumerate(train_loader, 1):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward and backward propagation
            outputs = model(inputs).squeeze(dim=1)
            loss = criteria(outputs, labels)
            loss.backward()
            optimizer.step()
        
        # Train loss and train accuracy
        train_current_loss, train_current_accuracy = validation(model, device, train_loader, criteria)

        # Early stopping
        current_loss, current_accuracy = validation(model, device, valid_loader, criteria)
        print('[{}/{} | Train loss: {:.8} | Train accuracy: {:.8} | Test loss: {:.8} | Test accuracy: {:.8}]'.format(epoch, epochs, train_current_loss, train_current_accuracy, current_loss, current_accuracy))
        
        train_loss_track.append(train_current_loss)
        train_accuracy_track.append(train_current_accuracy)
        test_loss_track.append(current_loss)
        test_accuracy_track.append(current_accuracy)
        
        if current_loss > last_loss:
            trigger_times += 1
            print('Trigger times:', trigger_times)

            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model

        else:
            print('Trigger times: 0')
            trigger_times = 0

        last_loss = current_loss
        
        if trigger_times == 0:
            print('New checkpoint...')
            checkpoint(epoch, model, optimizer, criteria, save_as=save_as)

    return model, (train_loss_track, train_accuracy_track), (test_loss_track, test_accuracy_track)

In [144]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = NetV0(input_features=30)
epochs = 100
learning_rate = 0.001
batch_size = 256
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criteria = nn.BCEWithLogitsLoss()
resume = 1
v0_path = Path('../Output/NetV0.pth')

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

recover = False

if v0_path.exists() and recover:
    print('Recovering old training process...')
    checkpoint = torch.load(v0_path)
    resume = checkpoint['epoch']
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    criteria = checkpoint['criteria']
else:
    print('New training process... =)')

New training process... =)


In [ ]:
trained_model, train_stats, test_stats = train(device, model, epochs, optimizer, criteria, train_loader, test_loader, resume=resume, save_as=v0_path)

[1/1000 | Train loss: 161.75128 | Train accuracy: 0.63953034 | Test loss: 161.9159 | Test accuracy: 0.63787129]
Trigger times: 1
[2/1000 | Train loss: 152.26798 | Train accuracy: 0.68091166 | Test loss: 152.33664 | Test accuracy: 0.68103143]
Trigger times: 0
New checkpoint...
[3/1000 | Train loss: 142.63881 | Train accuracy: 0.71794585 | Test loss: 142.79468 | Test accuracy: 0.71694883]
Trigger times: 0
New checkpoint...
[4/1000 | Train loss: 143.54637 | Train accuracy: 0.71339682 | Test loss: 143.86779 | Test accuracy: 0.71304763]
Trigger times: 1
[5/1000 | Train loss: 137.62164 | Train accuracy: 0.73494515 | Test loss: 137.82923 | Test accuracy: 0.73377997]
Trigger times: 0
New checkpoint...
[6/1000 | Train loss: 126.06116 | Train accuracy: 0.76493503 | Test loss: 126.54347 | Test accuracy: 0.76456161]
Trigger times: 0
New checkpoint...
[7/1000 | Train loss: 116.9635 | Train accuracy: 0.78994595 | Test loss: 117.30675 | Test accuracy: 0.789154]
Trigger times: 0
New checkpoint...
[8/1

## One hot encode f_27

In [7]:
X, y = df.drop(['target', 'id'], axis=1, errors='ignore'), df['target']
X.shape, y.shape

((810000, 31), (810000,))

In [8]:
f_27 = X['f_27']
f_27.nunique(), len(f_27)

(678113, 810000)

In [9]:
from sklearn.preprocessing import OneHotEncoder

X['f_27'] = X['f_27'].astype('category')
drop_enc = OneHotEncoder(drop='first')
sparse_matrix = pd.DataFrame.sparse.from_spmatrix(data=drop_enc.fit_transform(X[['f_27']]), columns=drop_enc.get_feature_names_out())
sparse_matrix.shape

(810000, 678112)

In [10]:
type(sparse_matrix)

pandas.core.frame.DataFrame

In [12]:
X = X.drop('f_27', axis=1, errors='ignore')
X = pd.concat([X, sparse_matrix], axis=1)

In [13]:
X.shape

(810000, 678142)

In [ ]:
X = (X - X.min()) / (X.max() - X.min())
X.shape, y.shape